# FAQs

Here you will find a few frequently asked questions. If you don't see your question below, feel free to contact us. 

In [1]:
import warnings
warnings.filterwarnings('ignore')
import numpy as np
#picaso
from picaso import justdoit as jdi
from picaso import justplotit as jpi
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

## How do I load in target properties from Exo.Mast? 

In [2]:
nexsci = jdi.all_planets()

In [3]:
nexsci.head()

,pl_hostname,pl_letter,pl_name,pl_discmethod,pl_controvflag,pl_pnum,pl_orbper,pl_orbpererr1,pl_orbpererr2,pl_orbperlim,...,pl_nnotes,rowupdate,pl_facility,pl_trandur,pl_eqt,st_logg,st_metfe,st_j,st_h,st_k
0,Kepler-135,b,Kepler-135b,Transit,0.0,2.0,6.002530,0.000016,-0.000016,0.0,...,1.0,2014-05-14,Kepler,0.1647,NaN,4.27,NaN,11.722,11.508,11.456
1,Kepler-135,c,Kepler-135c,Transit,0.0,2.0,11.448708,0.000145,-0.000145,0.0,...,1.0,2014-05-14,Kepler,0.2103,NaN,4.27,NaN,11.722,11.508,11.456
2,Kepler-133,c,Kepler-133c,Transit,0.0,2.0,31.517586,0.000158,-0.000158,0.0,...,1.0,2014-05-14,Kepler,0.3235,NaN,4.19,NaN,11.680,11.399,11.320
3,Kepler-134,b,Kepler-134b,Transit,0.0,2.0,5.317429,0.000008,-0.000008,0.0,...,1.0,2014-05-14,Kepler,0.1209,NaN,4.31,NaN,11.260,10.984,10.951
4,Kepler-134,c,Kepler-134c,Transit,0.0,2.0,10.105785,0.000053,-0.000053,0.0,...,1.0,2014-05-14,Kepler,0.0858,NaN,4.31,NaN,11.260,10.984,10.951


In [4]:
#first isolate the row 
gj1214_row = nexsci.loc[nexsci['pl_hostname']=='GJ1214']

#add it to load planet function with opacity
opa = jdi.opannection(wave_range=[1,5])
gj1214 = jdi.load_planet(gj1214_row,opa )

Load planet function adds in: 
    - planet properties and stellar properties 
    - parameterized PT profile 

That means you still need to add in the chemistry and cloud parameters

In [5]:
gj1214.channon_grid_high() #adds in template chemistry from sonora
df = gj1214.spectrum(opa, calculation='transmission+thermal')

In [6]:
x,y = jdi.mean_regrid(df['wavenumber'], df['fpfs_thermal'],R=150)
plot = [jpi.spectrum(x,y,plot_width=400, y_axis_type='log',y_axis_label='FpFs')]
x,y = jdi.mean_regrid(df['wavenumber'], df['transit_depth'],R=150)
plot += [jpi.spectrum(x,y,plot_width=400,y_axis_label='(Rp/Rs)^2')]

jpi.show(jpi.row(plot))

## How do I access the pressure-temperature profile parameterizations?

In [7]:
#start by loading in some template properties
opa = jdi.opannection(wave_range=[1,5])
gj1214_row = nexsci.loc[nexsci['pl_hostname']=='GJ1214']
gj1214 = jdi.load_planet(gj1214_row,opa )

### How do the input parameters effect the parameterization?

In [8]:
fig = [jpi.figure(width=300, height=300, y_axis_type='log', y_range=[100,1e-6],x_range=[400,1700],
                  y_axis_label='Pressure(bars)',x_axis_label='Temperature (K)',
                 title='Effect of logg1')]
colors = jpi.magma(10)
for i, logg1 in enumerate(np.linspace(-2,2,10)):
    gj1214.guillot_pt(576.17, T_int=100, logg1=logg1, logKir=-1.5)
    fig[0].line(gj1214.inputs['atmosphere']['profile']['temperature'],
            gj1214.inputs['atmosphere']['profile']['pressure'],color=colors[i])

fig += [jpi.figure(width=300, height=300, y_axis_type='log', y_range=[100,1e-6],x_range=[400,1700], 
                 x_axis_label='Temperature (K)',title='Effect of LogKir')]
for i,logKir in enumerate(np.linspace(-2,2,10)):
    gj1214.guillot_pt(576.17, T_int=100, logg1=-2, logKir=logKir)
    fig[1].line(gj1214.inputs['atmosphere']['profile']['temperature'],
            gj1214.inputs['atmosphere']['profile']['pressure'], color=colors[i])
        
fig += [jpi.figure(width=300, height=300, y_axis_type='log', y_range=[100,1e-6],x_range=[400,1700], 
                 x_axis_label='Temperature (K)',title='Effect of Tint')]       
for i,T_int in enumerate(np.linspace(50,300,10)):
    gj1214.guillot_pt(576.17, T_int=T_int, logg1=-2, logKir=-2)
    fig[2].line(gj1214.inputs['atmosphere']['profile']['temperature'],
            gj1214.inputs['atmosphere']['profile']['pressure'], color=colors[i])

jpi.show(jpi.row(fig))
                

## How do I return the output from photon attenuation plot? 

In [9]:
#first isolate the row 
gj1214_row = nexsci.loc[nexsci['pl_hostname']=='GJ1214']

#add it to load planet function with opacity
opa = jdi.opannection(wave_range=[0.3,1])
gj1214 = jdi.load_planet(gj1214_row,opa )
gj1214.channon_grid_high()
df = gj1214.spectrum(opa, calculation='reflected',full_output=True)

In [10]:
plot,wave,at_pressures_gas,at_pressures_cld,at_pressures_ray = jpi.photon_attenuation(df['full_output'],
                                                                                     return_output=True)

In [11]:
#now you can make your own plot!
fig = jpi.figure(y_axis_type='log',y_range=[100,1e-1],height=250, width=500)
for i,iy,ilab in zip([0,1],[at_pressures_gas,at_pressures_ray ], ['Molecular','Rayleigh']):
    x,y = jpi.mean_regrid(1e4/wave,iy, R=150)
    fig.line(1e4/x,y,legend_label=ilab, line_width=4, color=jpi.Colorblind8[i])
jpi.show(fig)

## Is there a way to see a break down of opacity contribution by molecule?

There is currently not a direct way to return this output. However there is a `plot_opacity` function that will show you a breakdown of what is currently being used to get your full output. If you have a specific need for the output, submit an issue via Github and we can add this functionality.  

In [12]:
#first isolate the row 
gj1214_row = nexsci.loc[nexsci['pl_hostname']=='GJ1214']

#add it to load planet function with opacity
opa = jdi.opannection(wave_range=[3,4])
gj1214 = jdi.load_planet(gj1214_row,opa )
gj1214.channon_grid_high()

df = gj1214.spectrum(opa, calculation='thermal',plot_opacity=True)



## Can I use the phase angle function to specify a non-zero phase for thermal emission? 

The phase angle function computes the incoming and outgoing angles. However, non-zero phase functionality is specific to __reflected light observations only__. This might be confusing because of course it is possible to observe thermal emission of planet at non-zero phase. However, unlike reflected light, thermal emission radiates from the planet in all directions (regardless of phase angle). 

__Do not force the code to run at non-zero phase angles for thermal emission.__ 

## Has your question not been answered? Feel free to contact us! 